In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/Colab Notebooks/deep-learning-group"

/content/drive/My Drive/Colab Notebooks/deep-learning-group


In [0]:
import autoreload
%load_ext autoreload

In [0]:
%reload_ext autoreload

In [0]:
import glob
import cv2
import numpy as np
import os


from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, AveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from tensorflow.keras.callbacks import EarlyStopping

import helpers

In [0]:
DATA_DIR = '/content/drive/My Drive/Colab Notebooks/dataset'

miniBatchSize = 200
colorMode = 'rgb'
classMode = "categorical"
imgSize = 64


datagen = ImageDataGenerator(rotation_range=20,
                             width_shift_range=0.1,
                             height_shift_range=0.1, 
                             rescale=1./255,
                             zoom_range=0.3,
                             validation_split=0.3,
                             horizontal_flip=True)

In [31]:
train_generator = datagen.flow_from_directory(DATA_DIR,
                                            target_size=(imgSize, imgSize),
                                            color_mode=colorMode,
                                            class_mode=classMode,
                                            batch_size=miniBatchSize,
                                            subset='training')

validation_generator = datagen.flow_from_directory(DATA_DIR,
                                            target_size=(imgSize, imgSize),
                                            color_mode=colorMode,
                                            class_mode=classMode,
                                            batch_size=miniBatchSize,
                                            subset='validation')

Found 914 images belonging to 17 classes.
Found 384 images belonging to 17 classes.


**Simple CNN model**

In [0]:
cvsores_1 = []
def get_compiled_model_1(optimizer='adam', loss='categorical_crossentropy'):
  keras.backend.clear_session()
  model = Sequential([
                      Conv2D(32, activation='relu', kernel_size=3, input_shape=(imgSize, imgSize, 3)),
                      MaxPooling2D(pool_size=(2, 2)),
                      # Conv2D(64, activation='relu', kernel_size=3),
                      # MaxPooling2D(pool_size=(2, 2)),
                      # Conv2D(32, activation='relu', kernel_size=3),
                      # MaxPooling2D(pool_size=(2, 2)),
                      Conv2D(32, activation='relu', kernel_size=3),
                      MaxPooling2D(pool_size=(2, 2)),
                      Flatten(),
                      Dense(128, activation='relu'),
                      Dense(17, activation='softmax')
  ])
  model.compile(optimizer=optimizer,
                loss=loss,
                metrics=['accuracy'])
  return model

In [0]:
model_1 = get_compiled_model_1()

model_1.fit(train_generator,
            steps_per_epoch=30,
            epoch=10,
            validation_data=validation_generator)

 1/30 [>.............................] - ETA: 0s - loss: 2.8631 - accuracy: 0.0700

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


30/30 [==============================] - ETA: 0s - loss: 2.7136 - accuracy: 0.1765